Variable elimination is a method for exact inference on graphs: more specifically, it computes marginal probabilities for random variables.  For example, let's imagine that we're considering the case of the burglar alarm (the second one, in which we ignore Mary and instead add a node that depends on earthquake that corresponds to a radio announcement that an earthquake has occurred).  We model the joint distribution as
$$
P(B,E,A,J,R) = P(B)P(E)P(A|B,E)P(J|A)P(R|E).
$$
This is a DAG, and all it does is encode assumptions of conditional independence.  To use it quantiatively, we need to specify probability tables:
 

In [1]:
import numpy as np

def P_B(B):
    if B==1:
        return 1e-2
    else:
        return 0.99

def P_E(E):
    if E==1:
        return 1e-3
    else:
        return 0.999

def P_A(A,B,E):
    cpt = np.array([[1e-3,0.3],
                    [0.9,0.95]])
    if A==1:
        return cpt[B,E]
    else:
        return 1-cpt[B,E]

def P_J(J,A):
    cpt = np.array([0.01,0.9])
    if J==1:
        return cpt[A]
    else:
        return 1-cpt[A]

def P_R(R,E):
    cpt = np.array([1e-5,0.999])
    if R==1:
        return cpt[E]
    else:
        return 1-cpt[E]


 We can then multiply each of these components to form the joint probability distribution:

In [7]:
def joint(B,E,A,J,R):
    return P_B(B)*P_E(E)*P_A(A,B,E)*P_J(J,A)*P_R(R,E)

Now that we have this graph specified, we are interested in using it to answer questions.  For example, let's imagine that we wish to answer the question "given that John called, what is the probability of a burglary?" Stated probabilistically, this can be answered by:
$$
P(B=1|J=1) = \frac{P(B=1,J=1)}{P(J=1)}
$$
Thus we have two marginals to compute: the joint probability of John calling and a burgling occurring, and just the marginal probability of John calling.  We can find these values by summing over all the non-query variables for both the numerator and the denominator:
$$
P(B=1|J=1) = \frac{\sum_{e,a,r} P(B=1,E,A,J=1,R)}{\sum_{b,e,a,r} P(B,E,A,J=1,R)}.
$$
There is a very simple (albeit naive) way of computing this:  for both the numerator and the denominator, we can compute these sums explicitly:

In [8]:
import itertools
#              B    E     A    J    R
states_B_J = [[1],[0,1],[0,1],[1],[0,1]]
enumerated_states = itertools.product(*states_B_J)
numerator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the numerator: ",len(numerator_states)
numerator = sum(numerator_states)

#              B    E     A    J    R
states_J = [[0,1],[0,1],[0,1],[1],[0,1]]
enumerated_states = itertools.product(*states_J)
denominator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the denominator: ",len(denominator_states)
denominator = sum(denominator_states)

probability = numerator/denominator
print "P(B=1|J=1) =", probability


Number of evaluated states in the numerator:  8
Number of evaluated states in the denominator:  16
P(B=1|J=1) = 0.42341151567790375


We can, of course use this same brute force method to answer other queries about the state.  For example, what if John calls *and* the radio broadcasts a statement that there has been an earthquake.  What then is the probability that a burglary occurs?
$$
P(B=1|J=1,R=1) = \frac{P(B=1,J=1,R=1)}{P(J=1,R=1)}
$$

In [6]:
#                B    E     A    J   R
states_B_J_R = [[1],[0,1],[0,1],[1],[1]]
enumerated_states = itertools.product(*states_B_J_R)
numerator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the numerator: ",len(numerator_states)
numerator = sum(numerator_states)

#              B    E     A    J    R
states_J_R = [[0,1],[0,1],[0,1],[1],[1]]
enumerated_states = itertools.product(*states_J_R)
denominator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the denominator: ",len(denominator_states)
denominator = sum(denominator_states)

probability = numerator/denominator
print "P(B=1|J=1,R=1) =", probability

Number of evaluated states in the numerator:  4
Number of evaluated states in the denominator:  8
P(B=1|J=1,R=1) = 0.030519067886791876


Also, just out of curiosity: What is the probability of a burglar, given only that we hear about an earthquake on the radio?  We should be able to figure this out directly from our graph (and through intuition), but it is helpful to do the computation as well:

In [7]:
#                B    E     A    J   R
states_B_J_R = [[1],[0,1],[0,1],[0,1],[1]]
enumerated_states = itertools.product(*states_B_J_R)
numerator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the numerator: ",len(numerator_states)
numerator = sum(numerator_states)

#              B    E     A    J    R
states_J_R = [[0,1],[0,1],[0,1],[0,1],[1]]
enumerated_states = itertools.product(*states_J_R)
denominator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the denominator: ",len(denominator_states)
denominator = sum(denominator_states)

probability = numerator/denominator
print "P(B=1|R=1) =", probability

Number of evaluated states in the numerator:  8
Number of evaluated states in the denominator:  16
P(B=1|R=1) = 0.010000000000000004


Of course, without any other observations, $R$ is independent from $B$ because of the block imposed by unobserved $A$.  Note that this is *not* the same as saying what is the probability of a burglary given that $R=1$ and that John didn't call $J=0$: $J=0$ is an observation.

In [8]:
#                B    E     A    J   R
states_B_nJ_R = [[1],[0,1],[0,1],[0],[1]]
enumerated_states = itertools.product(*states_B_nJ_R)
numerator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the numerator: ",len(numerator_states)
numerator = sum(numerator_states)

#              B    E     A    J    R
states_nJ_R = [[0,1],[0,1],[0,1],[0],[1]]
enumerated_states = itertools.product(*states_nJ_R)
denominator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the denominator: ",len(denominator_states)
denominator = sum(denominator_states)

probability = numerator/denominator
print "P(B=1|J=0,R=1) =", probability

Number of evaluated states in the numerator:  4
Number of evaluated states in the denominator:  8
P(B=1|J=0,R=1) = 0.0020135453488519866


Because John not calling is a good indicator that the burglar alarm is not going off, and the burglar alarm not going off is a good indicator of no burglar, this particular set of circumstances explains away much of the possibility of a burglar.  Conversely, it means that an earthquake is likely:

In [9]:
#                B    E     A    J   R
states_E_nJ_R = [[0,1],[1],[0,1],[0],[1]]
enumerated_states = itertools.product(*states_E_nJ_R)
numerator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the numerator: ",len(numerator_states)
numerator = sum(numerator_states)

#              B    E     A    J    R
states_nJ_R = [[0,1],[0,1],[0,1],[0],[1]]
enumerated_states = itertools.product(*states_nJ_R)
denominator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the denominator: ",len(denominator_states)
denominator = sum(denominator_states)

probability = numerator/denominator
print "P(E=1|J=0,R=1) =", probability

Number of evaluated states in the numerator:  4
Number of evaluated states in the denominator:  8
P(E=1|J=0,R=1) = 0.9865051761574405


Thus the combination of John not calling plus the earthquake announcement implies a 98.6% chance of an earthquake having occurred.

Note that the number of evaluated states in this case occurs in powers of two.  In particular, the number of states that we have to sum over is given by
$$
N_{s} = 2^{n-v}
$$
where $n$ is the number of variables and $v$ is the number of variables that we are not marginalizing over.  Thus for five variables and one query variable, we have to sum over $2^4=16$ states.  However, there is a better way to do this which is an example of the general technique of *dynamic programming*.  This technique is called *variable elimination*.  VE works by collapsing the graph into a smaller graph by sequentially marginalizing over leaves.  It's one of only a handful of problems that's more easily understood through equations.  Let's recall our joint distribution:
$$
P(B,E,A,J,R) = P(B)P(E)P(A|B,E)P(J|A)P(R|E).
$$
Let's imagine that we wish to compute the marginal distribution $P(J=1)$.  We then must sum over all the other variables.
$$
P(J=1) = \sum_b \sum_e \sum_a \sum_r P(B)P(E)P(A|B,E)P(J=1|A)P(R|E).
$$
Obviously, some of the summands are independent of the summation variable.  We can rearrange to get the following:
$$
P(J=1) = \sum_b P(B) \sum_e P(E) \sum_a P(A|B,E)P(J=1|A) \sum_r P(R|E),
$$
This rearrangement is useful for two reasons.  First, the final sum is equal to one:  we are just summing over the two possible values of $R$, which must sum to one.  This is called a *barren node*, and identifying and removing them gives us some nice simplification:
$$
P(J=1) = \sum_b P(B) \sum_e P(E) \sum_a P(A|B,E)P(J=1|A).
$$
Now, let's replace the final sum with a new factor:
$$
\gamma_a(B,E) = \sum_a P(A|B,E) P(J=1|A).
$$
An important note is that we're not just making this replacement symbolically: instead we are computing a new conditional probability table associated with $\gamma_a(B,E)$.  Thus, after we compute this once, we won't need to compute it again.  For example:


In [2]:
new_table_0 = np.array([[P_A(0,0,0)*P_J(1,0) + P_A(1,0,0)*P_J(1,1), P_A(0,0,1)*P_J(1,0) + P_A(1,0,1)*P_J(1,1)], 
                      [P_A(0,1,0)*P_J(1,0) + P_A(1,1,0)*P_J(1,1), P_A(0,1,1)*P_J(1,0) + P_A(1,1,1)*P_J(1,1)]])
def gamma_A(B,E):
    ptable = new_table_0
    return ptable[B,E] 


Making this substitution, we have
$$
P(J=1) = \sum_b P(B) \sum_e P(E) \gamma_a(B,E).
$$

Now, we can generate a similar factor:
$$
\gamma_e(B) = \sum_e P(E) \gamma_a(B,E)
$$

In [3]:
new_table_1 = np.array([P_E(0)*gamma_A(0,0) + P_E(1)*gamma_A(0,1),P_E(0)*gamma_A(1,0) + P_E(1)*gamma_A(1,1)])

def gamma_E(B):
    ptable = new_table_1
    return ptable[B]
    
gamma_E(0)

0.01115611

which gives us
$$
P(J=1) = \sum_b P(B) \gamma_e(B).
$$
We can easily compute the value of this:

In [4]:
print 'P(J=1) = ', P_B(0)*gamma_E(0) + P_B(1)*gamma_E(1)

P(J=1) =  0.0191549939


Comparing this to the brute force method:

In [9]:
#                B    E     A    J   R
states = [[0,1],[0,1],[0,1],[1],[0,1]]
enumerated_states = itertools.product(*states)
numerator_states = [joint(*x) for x in enumerated_states]
print "Number of evaluated states in the numerator: ",len(numerator_states)
numerator = sum(numerator_states)
print numerator

Number of evaluated states in the numerator:  16
0.019154993900000004


The results are identical.  However, If count the number of floating point operations, we'll see that for the brute force method, we had to compute $4 \times 16 + 16 = 80$ operations.  Using VE, we had to compute $3\times 4 + 3\times 2 + 3 = 21$ operations (plus a little bit of extra overhead assocated with storing the new variables.  This is a big savings.  While the brute force method is exponential in the number of nodes, VE is exponential in the maximum number of parental nodes.  Thus, for this graph, the *bottleneck* comes from the $A$ node, which has two parents.

Obviously, for graphs with lots of connections (i.e. with nodes containing lots of parents), this technique is still going to be quite slow.  However, it also implies that we can perform inference in linear time if all nodes are restricted to having one parent.  This type of graph has a special name: *a tree*.  